<a href="https://colab.research.google.com/github/Suchi0506/RFAssignment_Quiz/blob/main/RFAssignmentQuiz_Intellipat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded= files.upload()

Saving census-income.csv to census-income (2).csv


In [3]:
df=pd.read_csv('census-income.csv')

In [4]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Unnamed: 14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
#Replacing column 'Unnamed: 14' to 'income'
df.rename(columns={'Unnamed: 14':'income'},inplace=True)

In [6]:
# Assuming the dataset is already loaded into a DataFrame called `df`
percentage = (df[df['income'] == '>50K']['income'].count() / df['income'].count()) * 100

# Print the result
print(f"Percentage of population with income greater than 50K: {percentage:.2f}%")


Percentage of population with income greater than 50K: 24.08%


In [7]:
# Computing how many samples of the population are never married and working hours less than 20 hours
result = df[(df['marital-status'] == 'Never-married') & (df['hours-per-week'] < 20)]

count= result.shape[0]
count

907

In [8]:
# the correct list of age with minimum age , maximum and 50th Percentile of the age group
df['age'].describe()

,age
count,32561.000000
mean,38.581647
std,13.640433
min,17.000000
25%,28.000000
50%,37.000000
75%,48.000000
max,90.000000


In [9]:
# country having the highest population and  the lowest population
country_counts= df['native-country'].value_counts()
country_counts

highest_population_country= country_counts.idxmax()
lowest_population_country= country_counts.idxmin()

print(f"Country with the highest population: {highest_population_country}")
print(f"Country with the lowest population: {lowest_population_country}")

Country with the highest population: United-States
Country with the lowest population: Holand-Netherlands


In [14]:
#Select the best hyperparameters by RandomSearchCV and fit the model with the best hyperparameters and compute the accuracy score of the model.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# Applying one hot encoding
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_>50K
0,39,77516,13,2174,0,40,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,50,83311,13,0,0,13,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,38,215646,9,0,0,40,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,53,234721,7,0,0,40,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
4,28,338409,13,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [15]:
X = df_encoded.drop(columns='income_>50K', axis=1)
y = df_encoded['income_>50K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42)

# Setting up the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=rf, param_distributions=param_dist,
    n_iter=100, cv=3, verbose=2, random_state=42,
    n_jobs=-1, error_score='raise'
)

# Fitting the RandomizedSearchCV model
random_search.fit(X_train, y_train)

# Getting the best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Predicting using the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Computing the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Score: {accuracy}")


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 5, 'max_depth': 20}
Accuracy Score: 0.8667280822969445
